In [1]:
'''
context와 쿼리의 조사, 어미를 제거한후 tfidf를 만듭니다.
()로 씌어진건 ()를 띄어쓰기로 치환 《는 앞뒤에 띄어쓰기 없으면 추가, " '는 앞뒤로 띄어쓰기없으면 추가
고유명사의 특성을 살리기 위해 토크나이저는 띄어쓰기만 사용합니다.
유니그램 ? 바이그램 ? 
'''

'\ncontext와 쿼리의 조사, 어미를 제거한후 tfidf를 만듭니다.\n유니그램 ? 바이그램 ? \n'

In [198]:
import json
import os
import pickle
import time
from contextlib import contextmanager
from typing import List, NoReturn, Optional, Tuple, Union

import faiss
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)

In [222]:
data_path = "../../data"
context_path = "wikipedia_documents.json"
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contextss = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)

In [216]:
a = pd.read_csv("./checkcheck.csv")
a = list(a['question'])
qquery = a

In [125]:
f= open("./EOMI.TXT", 'r', encoding='cp949')
ff= open("./JOSA.TXT", 'r', encoding='cp949')

In [126]:
eomi = []
while 1:
    line = f.readline()
    if not line:
        break
    a= line[:-1]+ " "
    eomi.append(a)


In [127]:
josa = []
while 1:
    line = ff.readline()
    if not line:
        break
    a= line[:-1]+" "
    josa.append(a)

In [129]:
eomi = eomi[:-1]
eomi.append("었다 ")
eomi.append("있다 ")
eomi.append("켰다 ")
eomi.append("았다 ")
eomi.append("였다 ")
eomi.append("했다 ")
eomi.append("하였다 ")
eomi.append("된다 ")
eomi.append("한다 ")
eomi.append("혔다 ")
eomi.append("는다 ")
eomi.append("었다")
eomi.append("있다")
eomi.append("켰다")
eomi.append("았다")
eomi.append("였다")
eomi.append("했다")
eomi.append("하였다")
eomi.append("된다")
eomi.append("한다")
eomi.append("혔다")
eomi.append("는다")
eomi.sort(key=lambda x: -len(x))


In [130]:
josa.sort(key=lambda x: -len(x))
josa = josa[:-1]

In [131]:
for i in josa:
    if i not in eomi:
        eomi.append(i)

In [132]:
eomi.sort(key=lambda x: -len(x))

In [133]:
for i, v in enumerate(contexts):
    contexts[i] = contexts[i].replace(", ", " ")
    contexts[i] = contexts[i].replace(". ", " ")
    contexts[i] = contexts[i].replace(".", "")
    contexts[i] = contexts[i].replace("\n", " ")
    contexts[i] = contexts[i].replace(". ", " ")
    contexts[i] = contexts[i].replace(")", " ")
    contexts[i] = contexts[i].replace("(", " ")
    contexts[i] = contexts[i].replace(") ", " ")
    contexts[i] = contexts[i].replace("( ", " ")
    contexts[i] = contexts[i].replace("《", " 《")
    contexts[i] = contexts[i].replace("》", "》 ")
    contexts[i] = contexts[i].replace("*", " ")
    contexts[i] = contexts[i].replace("#", " ")
    
    

In [134]:
for i, v in enumerate(contexts):
    for ii in eomi:
        contexts[i] = contexts[i].replace(ii, " ")

In [142]:
for i, v in enumerate(query):
    query[i] = query[i].replace(", ", " ")
    query[i] = query[i].replace(". ", " ")
    query[i] = query[i].replace(".", "")
    query[i] = query[i].replace("\n", " ")
    query[i] = query[i].replace(". ", " ")
    query[i] = query[i].replace(")", " ")
    query[i] = query[i].replace("(", " ")
    query[i] = query[i].replace(") ", " ")
    query[i] = query[i].replace("( ", " ")
    query[i] = query[i].replace("《", " 《")
    query[i] = query[i].replace("》", "》 ")
    query[i] = query[i].replace("*", " ")
    query[i] = query[i].replace("#", " ")
    
for i, v in enumerate(query):
    for ii in eomi:
        query[i] = query[i].replace(ii, " ")

In [143]:
for i, v in enumerate(query):
    for ii in range(len(v)-1,0,-1):
        if query[i][ii] == " ":
            query[i] = query[i][0:ii]
            break
        

In [203]:
#def tokenizer(n):
#    n = n.split(" ")
#    return n
tokenizer = AutoTokenizer.from_pretrained(
        "klue/bert-base",
        use_fast=True,
    )

    

tfidf = TfidfVectorizer(
            tokenizer=tokenizer.tokenize, ngram_range=(1, 2), max_features=10000000,
        )

In [204]:
context_vec = tfidf.fit_transform(contexts)
query_vec = tfidf.transform(query)

Token indices sequence length is longer than the specified maximum sequence length for this model (675 > 512). Running this sequence through the model will result in indexing errors


In [153]:
a = pd.read_csv("./checkcheck.csv")
a= list(a['original_context'])
tmp = []
for v in a:
    for ii,vv in enumerate(contexts):
        if v == vv:
            tmp.append(ii)

In [211]:
    
    result = query_vec * context_vec.T
    for k in range(0,100,5):
        doc_scores = []
        doc_indices = []
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        ccnt = 0
        indice = 0
        index = []
        for i, ii in zip(tmp, doc_indices):
            if i in ii:
             ccnt+=1
            else:
             index.append(indice)
            indice+=1
        print("top ",k," acc: ", ccnt/240*100)

top  0  acc:  0.0
top  5  acc:  78.33333333333333
top  10  acc:  82.5
top  15  acc:  84.58333333333333
top  20  acc:  86.25
top  25  acc:  87.08333333333333
top  30  acc:  89.16666666666667
top  35  acc:  90.0
top  40  acc:  92.08333333333333
top  45  acc:  92.5
top  50  acc:  93.33333333333333
top  55  acc:  93.75
top  60  acc:  94.58333333333333
top  65  acc:  95.0
top  70  acc:  95.41666666666667
top  75  acc:  95.41666666666667
top  80  acc:  95.41666666666667
top  85  acc:  95.41666666666667
top  90  acc:  95.41666666666667
top  95  acc:  95.83333333333334


In [205]:
df = pd.DataFrame(query_vec[1].T.todense(), index=tfidf.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(50))

           TF-IDF
남쪽 코     0.334081
처 연장     0.334081
노선 처     0.313503
##버 남쪽   0.313503
##보 ##콘  0.306878
코 ##보    0.292925
연장 ##된   0.261572
스카 ##버   0.242996
철 노선     0.224565
스카       0.189918
연장       0.156996
##콘      0.156806
마을       0.151324
노선       0.147573
남쪽       0.134589
철        0.132413
##그      0.113948
코        0.108522
##버      0.105360
처        0.094347
##보      0.093459
##된      0.056655
세기 큰     0.000000
세기 전형    0.000000
세기 트레이드  0.000000
세기 정부    0.000000
세기 조절    0.000000
세기 주파수   0.000000
세기 중요    0.000000
세기 증가    0.000000
세기 초전    0.000000
세기 초저    0.000000
세기 지     0.000000
세기 지구    0.000000
세기 지나    0.000000
세기 찍     0.000000
세기 창출    0.000000
세기 콜     0.000000
세기 카테    0.000000
세기 전반    0.000000
세기 충분히   0.000000
세기 크     0.000000
세기 일     0.000000
세기 전     0.000000
세기 언어    0.000000
세기 선수    0.000000
세기 세     0.000000
세기 세계    0.000000
세기 세기    0.000000
세기 수소    0.000000


In [179]:
tmp

[]

In [212]:
query

['처 부실 경영인 대한 보상 선고 받',
 '스카버 남쪽 코보콘그 마을 철 노선 처 연장된',
 '촌락 운영 위원 후보 이름 쓰기위해 사용된',
 '로타이르 백조 구하 위해 사용한',
 '의견 자유롭 나누 것 조직 내 어떤 관계',
 '1945년 쇼와천황 항복 선언 발표된 라디',
 '징금수 서양 자수 어떤 기법 같 기술',
 '다른  의사들 감염내 전문의들로 어떤 것 대해 조언',
 '루 14세 왕비 마리 테래 어느 나',
 '헤자즈 왕국 실존했 것',
 '버드 교장 5월 여왕 대안 제시한',
 "인형사' 만들어낸 것 추측 사업",
 '멘데스 요원들 구하 위해 간 도시',
 '교과부 행동 화 여러명 사직한 기구',
 '반대동맹 공산당 갈라서겠 얘기한 날',
 '피어슨 다시 의회 해산했',
 '몽케 죽 뒤 쿠릴타 대칸 지위 얻 사람',
 '이흥구 사법시험 이야기 기사로 작성한',
 '남북조 시대 이 씨 전쟁 발생했 때 생활했',
 '박지훈 1라운드 몇 순위',
 '데메카론 무엇 풍자하 이야기',
 '병 걸 죽 확률 약 25~50% 달하 유형',
 '설리반 불 표시한 대상',
 '베소스 어디서',
 '진전사 명칭 드러나  영향 준',
 '자신 이상적인 국가관 스파르타 닮 있 생각하 플라톤',
 '박제된 북극곰 사망한',
 '문법 측 더 보수적인 포르투갈',
 '로스 수장 살해한 사람 어느 당',
 '조경숙왕 아들인 요자 친어머',
 '오래플린 부스 마지막 계획 따르 그들 어디서 링컨 납치하',
 '김득황 친일파로 취급되었 것 무슨 경력',
 '레닌 출간한 책 중 농민 자발적 참 대한 내용 포함되어있',
 '신란 동반자 죽었 전해',
 '칭자오 머리 엄청난 위력 발휘할 수 없 된 것 누구',
 '해초 조류 표  유기체 중 가장 비율 높',
 '류한욱 두 번째 뇌출혈로 쓰러',
 '로마 공성무 대한 기록 남',
 '동반 등록제 최초로 실시한 중국',
 '장대호 사용한',
 '스뮈츠 학비 지원해 준',
 '브루투스 세운도시 현재',
 '일본 대학 입시 며칠간',
 

In [215]:
josaa = [ '이 ','의 ','와 ','에 ', '만 ', '를 ', '랑 ', '도 ', '과 ', '가 ', '는다 ', '혔다 ', '한다 ', '된다 ', '했다 ', '였다 ', '았다 ', '켰다 ', '있다 ', '었다 ','을 ', '은 ']

In [217]:
for i, v in enumerate(qquery):
    qquery[i] = qquery[i].replace(", ", " ")
    qquery[i] = qquery[i].replace(". ", " ")
    qquery[i] = qquery[i].replace(".", "")
    qquery[i] = qquery[i].replace("\n", " ")
    qquery[i] = qquery[i].replace(". ", " ")
    qquery[i] = qquery[i].replace(")", " ")
    qquery[i] = qquery[i].replace("(", " ")
    qquery[i] = qquery[i].replace(") ", " ")
    qquery[i] = qquery[i].replace("( ", " ")
    qquery[i] = qquery[i].replace("《", " 《")
    qquery[i] = qquery[i].replace("》", "》 ")
    qquery[i] = qquery[i].replace("*", " ")
    qquery[i] = qquery[i].replace("#", " ")
    
for i, v in enumerate(qquery):
    for ii in josaa:
        qquery[i] = qquery[i].replace(ii, " ")

In [219]:
for i, v in enumerate(qquery):
    for ii in range(len(v)-1,0,-1):
        if qquery[i][ii] == " ":
            qquery[i] = qquery[i][0:ii]
            break

In [224]:
for i, v in enumerate(contextss):
    contextss[i] = contextss[i].replace(", ", " ")
    contextss[i] = contextss[i].replace(". ", " ")
    contextss[i] = contextss[i].replace(".", "")
    contextss[i] = contextss[i].replace("\n", " ")
    contextss[i] = contextss[i].replace(". ", " ")
    contextss[i] = contextss[i].replace(")", " ")
    contextss[i] = contextss[i].replace("(", " ")
    contextss[i] = contextss[i].replace(") ", " ")
    contextss[i] = contextss[i].replace("( ", " ")
    contextss[i] = contextss[i].replace("《", " 《")
    contextss[i] = contextss[i].replace("》", "》 ")
    contextss[i] = contextss[i].replace("*", " ")
    contextss[i] = contextss[i].replace("#", " ")

In [225]:
for i, v in enumerate(contextss):
    for ii in josaa:
        contextss[i] = contextss[i].replace(ii, " ")

In [228]:
contextss_vec = tfidf.fit_transform(contextss)
qquery_vec = tfidf.transform(qquery)

Token indices sequence length is longer than the specified maximum sequence length for this model (833 > 512). Running this sequence through the model will result in indexing errors


In [229]:
    result = qquery_vec * contextss_vec.T
    for k in range(0,100,5):
        doc_scores = []
        doc_indices = []
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        ccnt = 0
        indice = 0
        index = []
        for i, ii in zip(tmp, doc_indices):
            if i in ii:
             ccnt+=1
            else:
             index.append(indice)
            indice+=1
        print("top ",k," acc: ", ccnt/240*100)

top  0  acc:  0.0
top  5  acc:  78.75
top  10  acc:  84.16666666666667
top  15  acc:  85.83333333333333
top  20  acc:  87.08333333333333
top  25  acc:  88.33333333333333
top  30  acc:  88.33333333333333
top  35  acc:  89.58333333333334
top  40  acc:  91.66666666666666
top  45  acc:  92.5
top  50  acc:  93.33333333333333
top  55  acc:  93.33333333333333
top  60  acc:  94.16666666666667
top  65  acc:  95.0
top  70  acc:  95.41666666666667
top  75  acc:  95.41666666666667
top  80  acc:  95.41666666666667
top  85  acc:  95.41666666666667
top  90  acc:  96.25
top  95  acc:  96.25


In [230]:
qquery

['처음으로 부실 경영인 대한 보상 선고 받',
 '스카버러 남쪽 코보콘그 마 철 노선 처음 연장된',
 '촌락에서 운영 위원 후보자 이름 쓰기위해 사용된',
 '로타이르 백조 구하기 위해 사용한',
 '의견 자유롭게 나누는 것 조직 내 어떤 관계에서',
 '1945년 쇼와천황 항복 선언 발표된 라디오',
 '징금수는 서양 자수 어떤 기법 같 기술',
 '다른  의사들 감염내 전문의들로부터 어떤 것 대해 조언',
 '루 14세 왕비 마리아 테래사는 어느 나라',
 '헤자즈 왕국 실존했던 것',
 '버드 교장 5월 여왕 대안으로 제시한',
 "인형사' 만들어낸 것으로 추측되는 사업",
 '멘데스 요원들 구하기 위해 간 도시는',
 '교과부 행동 화가나 여러명 사직한 기구',
 '반대동맹 공산당 갈라서겠다고 얘기한 날',
 '피어슨 다시 의회 해산했던',
 '몽케 죽 뒤 쿠릴타이에서 대칸 지위 얻 사람',
 '이흥구 사법시험 이야기 기사로 작성한',
 '남북조 시대에서 이 씨 전쟁 발생했 때 생활했던',
 '박지훈 1라운드에서 몇 순위',
 '데메카론에는 무엇 풍자하는 이야기',
 '병 걸려 죽 확률 약 25~50% 달하는 유형',
 '설리반 불만 표시한 대상',
 '베소스는 어디서',
 '진전사 명칭 드러나는 데 영향 준',
 '자신 이상적인 국가관 스파르타 닮아 있다고 생각하는 플라톤',
 '박제된 북극곰 사망한',
 '문법 측면에서 더 보수적인 포르투갈어',
 '로스 수장 살해한 사람 어느 당',
 '조경숙왕 아들인 요자 친어머니는',
 '오래플린 부스 마지막 계획 따르면 그들 어디서 링컨 납치하려고',
 '김득황 친일파로 취급되었던 것 무슨 경력',
 '레닌 출간한 책 중 농민 자발적 참여 대한 내용 포함되어있는',
 '신란 동반자 죽었다고 전해지는',
 '칭자오 머리 엄청난 위력 발휘할 수 없게 된 것 누구',
 '해초나 조류 표면에서 자라는 유기체 중 가장 비율 높',
 '류한욱 두 번째 뇌출혈로 쓰러진',
 '로마 공성무기 대한 기록 남긴',
 '동반